## Задание

- Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
- Cохранить результаты в pandas dataframe

---

Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных: 
 * Привести колонки датафрейма к одному типу данных 
 * Проверить индекс, чтобы он соответствовал порядку строк 
 * Проверить наличие пропусков и в случае обнаружения, определить стратегию их обработки

---

Провести мини-исследование рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов. 
+ Рассчитать средние цены за квадратный метр по регионам. Важно чтобы пропуски не учитывались при расчете средней цены. 
+ Рассчитать объемы вводимого жилья по регионам и по годам. 
+ Сравнить этажность домов, количество квартир, жилую площадь, количество парковочных мест и цену за квадратный метр в разных регионах 
+ Визуализировать разброс, связь и распределение этих показателей по регионам 
+ Сделать выводы по каждому из расчетов и графиков

### Загрузка библиотек

In [39]:
import requests
import pandas as pd

In [38]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 150
pd.options.display.precision = 2
pd.options.display.show_dimensions = True
pd.options.display.precision = 6

### Сбор информации

In [ ]:
# Получение списка объектов

limit_ = 1000
all_obj_ids = []
for offset_ in range(0, 12000, 1000):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    response = requests.get(url)
    objects_data = response.json()
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    all_obj_ids.extend(objids)

In [ ]:
len(all_obj_ids)

In [ ]:
# Сбор информации по объектам

all_objects_description = []

for id in all_obj_ids:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
    response_object = requests.get(url)
    obj = response_object.json()
    all_objects_description.append(obj)

### Загрузка DataFrame

In [40]:
# Список используемых колонок

use_columns = [
    'data.id', 'data.region','data.developer.devShortCleanNm','data.region',
   'data.address', 'data.objElemLivingCnt', 'data.objReady100PercDt', 'data.wallMaterialShortDesc',
   'data.objLkFinishTypeDesc', 'data.objLkFreePlanDesc', 'data.objElemParkingCnt',
   'data.objLkLatitude', 'data.objLkLongitude', 
   'data.objLkClassDesc', 'data.objFloorCnt', 'data.objFlatCnt',
   'data.objFlatSq', 'data.objNonlivElemCnt', 'data.objStatus',
   'data.objTransferPlanDt', 'data.objInfrstrBicycleLaneFlg', 
   'data.objInfrstrPlaygrndCnt', 'data.objInfrstrSportGroundCnt',
   'data.objInfrstrTrashAreaCnt', 'data.objInfrstrRampFlg', 
   'data.objInfrstrCurbLoweringFlg', 'data.objElevatorWheelchairCnt',
   'data.objElevatorPassengerCnt', 'data.objElevatorCargoCnt',
   'data.soldOutPerc', 'data.objPriceAvg', 'data.nonlivFirstFloor',
   'data.objEnergyEffShortDesc', 'data.infrastructureIndexValue',
   'data.quartography.objLivElem1KCnt', 'data.quartography.objLivElem2KCnt',
   'data.quartography.objLivElem3KCnt', 'data.quartography.objLivElem4KCnt',
   'data.quartography.objLivElemCnt', 'data.quartography.objLivElemEntrCnt',
   'data.quartography.objLivElemEntrFloorAvg'
]

In [41]:
# Словарь для переименования колонок
name_columns = {
    'data.id': 'Id_объекта',
   'data.developer.devShortCleanNm': 'Краткое наименование организации',
   'data.region': 'Регион строительства',
   'data.address': 'Адрес строительства',
   'data.objElemLivingCnt': 'Количество квартир',
   'data.objReady100PercDt': 'Ввод в эксплуатацию',
   'data.wallMaterialShortDesc': 'Материал стен',
   'data.objLkFinishTypeDesc': 'Тип отделки',
   'data.objLkFreePlanDesc': 'Свободная планировка',
   'data.objElemParkingCnt': 'Количество мест в паркинге',
   'data.objLkLatitude': 'Широта',
   'data.objLkLongitude': 'Долгота',
   'data.objLkClassDesc': 'Тип недвижимости',
   'data.objFloorCnt': 'Количество этажей',
   'data.objFlatCnt': 'Количество квартир',
   'data.objFlatSq': 'Жилая площадь, м²',
   'data.objNonlivElemCnt': 'Количество нежилых помещений',
   'data.objStatus': 'Статус строительства',
   'data.objTransferPlanDt': 'Выдача ключей',
   'data.objInfrstrBicycleLaneFlg': 'Велосипедные дорожки',
   'data.objInfrstrPlaygrndCnt': 'Количество детских площадок',
   'data.objInfrstrSportGroundCnt': 'Количество спортивных площадок',
   'data.objInfrstrTrashAreaCnt': 'Количество площадок для сбора мусора',
   'data.objInfrstrRampFlg': 'Наличие пандуса',
   'data.objInfrstrCurbLoweringFlg': 'Наличие понижающих площадок',
   'data.objElevatorWheelchairCnt': 'Количество инвалидных подъемников',
   'data.objElevatorPassengerCnt': 'Количество пассажирских лифтов',
   'data.objElevatorCargoCnt': 'Количество грузовых лифтов',
   'data.soldOutPerc': 'Распроданность квартир',
   'data.objPriceAvg': 'Средняя цена за 1 м²',
   'data.nonlivFirstFloor': 'Первый этаж нежилой',
   'data.objEnergyEffShortDesc': 'Класс энергоэффективности здания',
   'data.infrastructureIndexValue': 'Индекс доступности инфраструктуры',
   'data.quartography.objLivElem1KCnt' : 'Количество 1-комнатных квартир',
   'data.quartography.objLivElem2KCnt' : 'Количество 2-комнатных квартир',
   'data.quartography.objLivElem3KCnt' : 'Количество 3-комнатных квартир',
   'data.quartography.objLivElem4KCnt' : 'Количество 4-комнатных квартир',
   'data.quartography.objLivElemCnt' : 'Количество квартир',
   'data.quartography.objLivElemEntrCnt' : 'Количество подъездов',
   'data.quartography.objLivElemEntrFloorAvg' : 'Среднее количество квартир на этаже'
}

In [42]:
# data = pd.DataFrame.from_dict(pd.json_normalize(all_objects_description), orient='columns')
data = pd.read_csv('dom.csv', usecols=use_columns, parse_dates=['data.objReady100PercDt','data.objTransferPlanDt'])

In [43]:
data.rename(columns=name_columns, inplace=True)


In [44]:
data.head()

,Id_объекта,Краткое наименование организации,Регион строительства,Адрес строительства,Количество квартир,Ввод в эксплуатацию,Материал стен,Тип отделки,Свободная планировка,Количество мест в паркинге,Широта,Долгота,Тип недвижимости,Количество квартир,"Жилая площадь, м²",Количество нежилых помещений,Статус строительства,Велосипедные дорожки,Наличие пандуса,Наличие понижающих площадок,Класс энергоэффективности здания,Индекс доступности инфраструктуры,Количество этажей,Количество 1-комнатных квартир,Количество 2-комнатных квартир,Количество 3-комнатных квартир,Количество 4-комнатных квартир,Количество квартир,Количество подъездов,Среднее количество квартир на этаже,Выдача ключей,Количество детских площадок,Количество спортивных площадок,Количество площадок для сбора мусора,Количество инвалидных подъемников,Количество пассажирских лифтов,Количество грузовых лифтов,Распроданность квартир,Средняя цена за 1 м²,Первый этаж нежилой
0,29679,212-2,61,г Таганрог,18,2020-03-31,Другое,Под ключ,Есть,0,47.2238,38.8513,Комфорт,18,1519.00,0,0,0,0,0,Не нормируется,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29678,212-2,61,"город Таганрог, улица Сергея Шило, д. 212-2",27,2020-03-31,Другое,Под чистовую,Нет,0,47.2197,38.9134,Комфорт,27,1519.00,0,0,0,0,0,Не нормируется,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13383,2МЕН ГРУПП,72,г Тюмень,68,2022-12-31,Другое,По желанию заказчика,Есть,31,57.1519,65.5536,Элитный,68,6293.78,10,0,0,1,1,B,10,20.0,13.0,21.0,34.0,0.0,68.0,1.0,6.0,2023-02-28 12:00:00,1.0,1.0,1.0,0.0,5.0,1.0,0.407164,53109.0,1.0
3,13397,2МЕН ГРУПП,72,"г Тюмень, ул Профсоюзная, д. 56",65,2024-12-31,Другое,Без отделки,Нет,0,57.1519,65.5544,Элитный,65,6248.51,70,0,0,1,1,B,10,12.0,0.0,24.0,35.0,6.0,65.0,4.0,3.0,2025-02-28 12:00:00,1.0,1.0,1.0,0.0,5.0,4.0,0.999984,45523.0,0.0
4,13399,2МЕН ГРУПП,72,"г Тюмень, ул Профсоюзная, д. 56",0,2024-12-31,Другое,Без отделки,Нет,125,57.1519,65.5544,Элитный,0,0.00,0,0,0,1,1,B,10,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-02-28 12:00:00,0.0,0.0,1.0,0.0,8.0,7.0,0.917243,50768.0,1.0


In [ ]:
data[data['Регион строительства'] == 67]

In [ ]:
# Количество пропусков

data.isnull().sum().sort_values(ascending=False)